In [ ]:
%run -i datapy/analyze/classes.py
%run -i datapy/analyze/descriptives.py
%run -i datapy/analyze/recomendation.py
%run -i datapy/analyze/server.py

In [ ]:
igor_path='C:\\Users\\Admin\\Desktop\\strategy_full.csv'
#igor_path='C:\\Users\\shcherbakov2-ia\\Documents\\Python Scripts\\other\\prj\\strategy_full.csv'
anton_path='C:\\Users\\Anton\\repository\\datapy\\data\\strategy_full.csv'
path=igor_path

In [ ]:
%run -i datapy/analyze/server.py
s=server()
s.setfile(path,delimiter=';', decimal=',',encoding='1251')

## Определение ролей

In [ ]:
# начальное
types_and_roles, missing= s.PredictTypesAndRoles()
missing

In [ ]:
for i in range(len(types_and_roles)):
    print(i, types_and_roles[i].name, types_and_roles[i].type, types_and_roles[i].role)

In [ ]:
#Пользовательская корректировка

types_and_roles[0].role = VariableRoleEnum.ID
types_and_roles[1].role = VariableRoleEnum.DATE
for i in range(2, 21):
    types_and_roles[i].role = VariableRoleEnum.REJECTED
for i in range(22, 35):
    types_and_roles[i].role = VariableRoleEnum.FIXED_RULE
for i in range(35, 39):
    types_and_roles[i].role = VariableRoleEnum.HISTORICAL_RULE
for i in range(39, 41):
    types_and_roles[i].role = VariableRoleEnum.POTENCIAL_RULE


types_and_roles[21].role = VariableRoleEnum.TARGET
types_and_roles[20].role = VariableRoleEnum.NEW_CREDIT
types_and_roles[41].role = VariableRoleEnum.APPROVED
types_and_roles[42].role = VariableRoleEnum.NTU
types_and_roles[43].role = VariableRoleEnum.CREDITED

for i in range(44,len(types_and_roles)): 
    types_and_roles[i].role = VariableRoleEnum.REJECTED

for i in range(51,len(types_and_roles)): 
    types_and_roles[i].role = VariableRoleEnum.VALUE_COLUMN

types_and_roles_edited=types_and_roles
data_desctiption, data_desctiption_m, one_factor, unique_factor, time, ch_rules, p_rule, const_columns, val_columns = s.AnalyseMarkedData(types_and_roles_edited)
#вывод таблицы с расчетным временем
time

In [ ]:
#вывод таблицы со статистикой
data_desctiption

In [ ]:
#вывод таблицы со статистикой
data_desctiption_m

In [ ]:
#вывод таблицы с однофакторным анализом
one_factor

In [ ]:
#вывод таблицы с анализом уникальных отказов
unique_factor

In [ ]:
#удаляем столбцы с количеством пропусков выше заданного. по дефолту стоит 5%, можно установить свое значение
missing_threshold=0.9
s.DeleteMissing(missing_threshold)

# Пользователь выбирает параметры
после этого должен быть шаг выбора параметров:
опция 1: максимальный возможный уровень одобрения, при уровне риска не выше X% (уровень риска по одобренным или уровень риска по выданным - в зависимости от того, предоставлено ли поле NTU)
опция 2: минимальный уровень риска, при уровне одобрения не ниже Y%

In [ ]:
#поиск всех возможных комбинаций
found_combinations = s.FindCombinations()

In [ ]:
#поиск комбинаций по параметрам
# 1: максимальный возможный уровень одобрения, при уровне риска не выше X% (уровень риска по одобренным
param_max_dr=15
optimized_by_dr=s.OptimezeRulesDR(param_max_dr)
dr = pd.DataFrame(optimized_by_dr)

# 2: минимальный уровень риска, при уровне одобрения не ниже Y%
param_min_ar=10
optimized_by_ar=s.OptimezeRulesAR(param_min_ar)
ar = pd.DataFrame(optimized_by_ar)

In [ ]:
dr

# Пользователь вводит своё собственное правило

In [ ]:
users_combination=['r_limit_loanamount', 'r_limit_loanburden', 'r_verification', 'r_security']
user_combs=s.FindUsersCombination(users_combination)

In [ ]:
user_combs.head(1)

# Поиск однофакторных правил

In [ ]:
# метод для визуализации найденных правил
def RuleToText(rule):
    column_name=rule[0]
    thr=rule[1]
    trend=rule[2]
    dr_delta=rule[3]
    ar_delta=rule[4]
    if trend is None:
        return "Для поля {} не было найдено подходящих правил".format(column_name)
    elif trend:
        return "Рекомендуется ограничить выдачи по тем заявкам, где {} выше {}. Что позволит снизить DR на {}%, изменение AR составит: {}%".format(column_name, thr, round(dr_delta*100,2), round(ar_delta*100,2))
    else:
        return "Рекомендуется ограничить выдачи по тем заявкам, где {} ниже {}. Что позволит снизить DR на {}%, изменение AR составит: {}%".format(column_name, thr, round(dr_delta*100,2), round(ar_delta*100,2))

In [ ]:
user_columns=['CalcCalculatedMaxLoanAmount' ]
user_onefactor_rules=s.RecomendOneFactorRules(user_columns)
for rule in user_onefactor_rules:
    print(RuleToText(rule))

In [ ]:
#Если пользователь не передал ничего - анализируем всё
full_onefactor_rules=s.RecomendOneFactorRules()

In [ ]:
for rule in full_onefactor_rules:
    print(RuleToText(rule))

In [ ]:
#Если пользователь передал набор полей - анализируем по ним
user_columns=['ClientJobTerm','ClientCreditPayment']
user_onefactor_rules=s.RecomendOneFactorRules(user_columns)

In [ ]:
for rule in user_onefactor_rules:
    print(RuleToText(rule))

In [ ]:
#поиск комбинаций по параметрам
# 1: максимальный возможный уровень одобрения, при уровне риска не выше X% (уровень риска по одобренным
param_max_dr=15
optimized_by_dr=s.OptimezeRulesDR(param_max_dr)
dr = pd.DataFrame(optimized_by_dr)

# 2: минимальный уровень риска, при уровне одобрения не ниже Y%
param_min_ar=10
optimized_by_ar=s.OptimezeRulesAR(param_min_ar)
ar = pd.DataFrame(optimized_by_ar)

In [ ]:
dr

# Поиск многофакторных правил

In [ ]:
# метод для визуализации найденных мульти-правил
def MultyRuleToText(rule, delta_dr):
    if rule is None:
        return 'Не было найдено подходящих правил. Проверьте, возможно один из столбцов пустой'
    result_text='Рекомендуется ограничить выдачи по заявкам, где '
    for r in rule[:-1]:
        result_text+='{} {} {} и '.format(r[0], r[1], round(r[2],4))
    result_text = result_text[:-3] + '. Что позволит снизить DR на {}%'.format(round(delta_dr*100, 2))
    return result_text

In [ ]:
new_val_columns = [v for v in val_columns if v in s._data.columns]

In [ ]:
# Кодируем категориальные столбцы, выкинув удалённые столбцы
encode_dic=s.EncodeCategories(new_val_columns)

In [ ]:
#Если пользователь не передал ничего - анализируем всё
best_multirule, dr_delta=s.RecomendMultyFactorRules()
MultyRuleToText(best_multirule, dr_delta)

In [ ]:
#Если пользователь передал набор полей - анализируем по ним
user_columns=['ClientJobTermTotal', 'ClientPaymentCards']
user_best_multirule, user_dr_delta=s.RecomendMultyFactorRules(user_columns)
MultyRuleToText(user_best_multirule, user_dr_delta)

# Создание новых бинарных правил

In [ ]:
#Применение рекомендованных правил
new_types_and_roles, potential_rules=s.Bin_new_rules(full_onefactor_rules)

In [ ]:
#пользовательские правила
user_input='ClientPaymentCards>1'
user_rules=s.ParseRule(user_input)
and_rules=[r[0] for r in user_rules]

new_types_and_roles, potential_rules=s.Bin_new_rules(and_rules)

# Однофакторные правила с выводом как в  Single factor analysis

In [ ]:
#поиск возможных правил
new_user_onefactor_rules=s.RecomendOneFactorRules()
for rule in new_user_onefactor_rules:
    print(RuleToText(rule))

In [ ]:
#из новых предложенных правил пользователь выбирает те, что хочет создать
new_user_rules_to_create=[new_user_onefactor_rules[0], new_user_onefactor_rules[2]]
#и создаёт их 
new_types_and_roles, potential_rules=s.Bin_new_rules(new_user_rules_to_create)

In [ ]:
#повторно получаем статистику, теперь уже с новыми маркерами
new_data_desctiption, new_data_desctiption_m, new_one_factor, new_unique_factor, new_time, new_ch_rules, new_p_rule, const_columns, val_columns = s.AnalyseMarkedData(new_types_and_roles)

In [ ]:
new_time

In [ ]:
#поиск всех возможных комбинаций
found_combinations =s.FindCombinations()
found_combinations

In [ ]:
def int_to_obj(data, dick):
    for k, val in dick.items():
        for v, n in val.items():
            data.loc[data[k] == n, k] = v